In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import t
import numpy as np
import plotly.express as px
import plotly.offline as pyo

In [27]:
pyo.init_notebook_mode()
# pip install plotly jupyterlab 
# (Install plotly jupyerlab and restart jupyter kernel)
# pip install jupyterlab-spreadsheet-editor

In [18]:
# mamba install -c conda-forge nodejs
# then install extensions

# Define Functions

In [19]:
def summarySE(data=None, measurevar=None, groupvars=None, na_rm=False, conf_interval=0.95):
    # New version of length which can handle NA's: if na_rm==True, don't count them
    def length2(x, na_rm=False):
        if na_rm:
            return np.sum(~np.isnan(x))
        else:
            return len(x)

    # This does the summary. For each group's data frame, return a DataFrame with N, mean, and sd
    def summary_fun(xx, col):
        return pd.Series({
            'N': length2(xx[col], na_rm=na_rm),
            'mean': np.mean(xx[col]),
            'sd': np.std(xx[col])
        })

    datac = data.groupby(groupvars, dropna=True).apply(summary_fun, col=measurevar).reset_index()

    # Rename the "mean" column
    datac.rename(columns={'mean': measurevar}, inplace=True)

    datac['se'] = datac['sd'] / np.sqrt(datac['N'])  # Calculate standard error of the mean

    # Confidence interval multiplier for standard error
    # Calculate t-statistic for confidence interval:
    # e.g., if conf_interval is 0.95, use 0.975 (above/below), and use df=N-1
    ci_mult = t.ppf(conf_interval / 2 + 0.5, datac['N'] - 1)
    datac['ci'] = datac['se'] * ci_mult

    return datac 




# # Example data (replace these paths with your actual data paths)
# L1_data = {
#     'subject': ['A', 'B', 'C'],
#     'support': [1, 2, 1],
#     'recall': [0.8, 0.6, 0.7]
# }

# L2_data = {
#     'subject': ['A', 'B', 'C'],
#     'support': [1, 2, 1],
#     'recall': [0.7, 0.5, 0.6]
# }

# L1 = pd.DataFrame(L1_data)
# L2 = pd.DataFrame(L2_data)

# LINE = pd.concat([L1, L2], ignore_index=True)


In [20]:
def plot_sensitivity_boxstrip(df, x_column, y_column,
                              title, palette):
    """
    Plot sensitivity using Seaborn with boxplot and stripplot.

    Parameters:
    - df (pd.DataFrame): DataFrame containing sensitivity data.
    - x_column (str): Column for the x-axis.
    - y_column (str): Column for the y-axis.
    - palette (list): Color palette for the plot.

    Returns:
    - None
    """

    plt.figure(figsize=(10, 6))

    # Boxplot
    sns.boxplot(x=x_column, y=y_column, data=df, hue="subject",
                width=0.5, linewidth=0.2, palette=palette, fliersize=0)

    # # Scatter plot with jitter and dodge
    sns.stripplot(x=x_column, y=y_column, data=df, hue="subject",
                  palette=palette, s=5, dodge=True, jitter=True,
                  alpha=0.7, edgecolor='black', linewidth=0.2)

    plt.xlabel("")
    plt.ylabel("")
    plt.ylim(0, 1.05)
    plt.yticks([0, 0.2, 0.4, 0.6, 0.8, 1])
    plt.xticks([0, 1], labels=[">=1 read", ">=2 reads"])

    # Remove the legend
    plt.legend().set_visible(False)

    # Set the color palette for fill and color
    sns.set_palette(palette)

    plt.grid(True, linestyle='--', alpha=0.7)

    plt.title(title)

    # Adjust font sizes
    plt.rc('xtick', labelsize=12)
    plt.rc('ytick', labelsize=12)
    plt.rc('axes', labelsize=24)

    plt.show()

## Plotly
# import plotly.express as px

# def plot_sensitivity_boxstrip(df, x_column, y_column, title, palette):
#     """
#     Plot sensitivity using Plotly Express with box plot and strip plot.

#     Parameters:
#     - df (pd.DataFrame): DataFrame containing sensitivity data.
#     - x_column (str): Column for the x-axis.
#     - y_column (str): Column for the y-axis.
#     - palette (list): Color palette for the plot.

#     Returns:
#     - None
#     """

#     # Create box plot with points
#     fig = px.box(df, x=x_column, y=y_column, color="subject",
#                   color_discrete_sequence=palette,points="all",
#                  notched=True, boxmode="group")

#     # Update layout for box plot
#     fig.update_traces(marker=dict(size=8, line=dict(width=1, color='black')))

#     # Update layout
#     fig.update_layout(
#         title=title,
#         xaxis_title="",
#         yaxis_title="",
#         xaxis=dict(tickmode='array', tickvals=[0, 1], ticktext=[">=1 read", ">=2 reads"]),
#         yaxis=dict(range=[0, 1.05], tickvals=[0, 0.2, 0.4, 0.6, 0.8, 1]),
#         font=dict(size=12),
#         legend=dict(title=""),
#         hovermode="closest",
#         hoverlabel=dict(bgcolor="white", font_size=12),
#     )

#     # Show the figure
#     fig.show()

In [21]:
def create_line_plot(df, x_column, y_column, x_label, y_label, title):
    """
    Create a line plot using Plotly Express.

    Parameters:
    - df (pd.DataFrame): DataFrame containing the data.
    - x_column (str): Column for the x-axis.
    - y_column (str): Column for the y-axis.
    - x_label (str): Label for the x-axis.
    - y_label (str): Label for the y-axis.
    - title (str): Title of the plot.

    Returns:
    - None
    """

    fig = px.line(df, x=x_column, y=y_column, color='Depth',
                  line_group='Depth', markers=True,
                  color_discrete_sequence=px.colors.qualitative.Set1,
                  symbol='Depth', symbol_sequence=[1, 3, 16, 17])

    fig.update_traces(marker=dict(size=10))

    fig.update_layout(
        xaxis_title=x_label,
        yaxis_title=y_label,
        xaxis=dict(
            type='log',
            tickvals=[0.04, 0.2, 1, 5, 25],
            range=[-2, 3.25]
        ),
        yaxis=dict(
            range=[0, 1.1],
            tickvals=[0, 0.2, 0.4, 0.6, 0.8, 1]
        ),
        legend=dict(
            traceorder='normal',
            orientation='v',
            y=1.1,
            x=0.01,
            font=dict(size=16),
            title=dict(text='Depth', font=dict(size=18, color='black'))
        ),
        font=dict(
            family="Arial, sans-serif",
            size=18,
            color="RebeccaPurple"
        ),
        title=title
    )

    fig.show()


# 1. Sensitivity, Precision, False Positive Plots

In [22]:
df_L1_data = pd.read_excel('exampleData/SD_Fig_2.xlsx', sheet_name='Fig2a_one_read')
df_L2_data = pd.read_excel('exampleData/SD_Fig_2.xlsx', sheet_name='Fig2a_two_reads')

In [23]:
LINE = pd.concat([df_L1_data, df_L2_data], ignore_index=True)
LINE['support'] = LINE['support'].str.extract('(\d+)', expand=False).astype(float)

In [24]:
LINE

In [25]:
# Define the color palette
cbPalette = ["#E41A1C", "#377EB8", "#4DAF4A"]

## 1.1. RetroSom

In [26]:
plot_sensitivity_boxstrip(LINE,"support","recall",
                          "Sensitivity",cbPalette)

plot_sensitivity_boxstrip(LINE,"support","precision",
                          "Precision",
                          cbPalette)

## 1.2. xTea

# 2. Sensitivity and Precision per Level of Mosaicism

In [12]:
dat = pd.read_excel('exampleData/SD_Fig_2.xlsx', sheet_name='Fig2c,d')

In [13]:
# Extract columns 1 to 3 for 'sens'
sens = dat.iloc[:, :3]

# Extract all columns except the third for 'prec'
#prec = dat.iloc[:, :-1]
prec = dat.drop(dat.columns[2], axis=1)

# Convert 'Depth' columns to factors
sens['Depth'] = sens['Depth'].astype('category')
prec['Depth'] = prec['Depth'].astype('category')

## 2.1. RetroSom

In [30]:
import matplotlib.pyplot as plt
%matplotlib inline

In [31]:
create_line_plot(sens, 'Freq', 'Sensitivity', "Level of Mosaicism (%)", "Sensitivity (L1)", "Sensitivity Plot")
create_line_plot(prec, 'Freq', 'Precision', "Level of Mosaicism (%)", "Precision (L1)", "Precision Plot")

## 2.2. xTea-Mosaic